# Build a variational autoencoder

Based on the ability to generate UK car registration numbers, one can build a dataset of training and test data. With a suitable vectorizer, one can go ahead and make an autoencoder.

The decoder part of the variational encoder can then be used to generate new reg numbers.

### References

https://blog.keras.io/building-autoencoders-in-keras.html

https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import backend
from keras import optimizers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegDvlaVectorizer

### Data generation

In [2]:
n_train = 10000
n_val = 1000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegDvlaVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
val_strs = [g.get_reg() for _ in range(n_val)]
val_strs = [x for x in val_strs if x not in set(train_strs)]  # sanity check
val_vecs = np.array([v.vectorize(x) for x in val_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs) | set(val_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of validation samples = ' + str(len(val_strs)))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 150
actual number of validation samples = 1000
actual number of test samples = 1000


### Variational autoencoder

Design the architecture

In [3]:
latent_dim = 15
original_dim = vec_length

### Define the encoder
inputs = keras.Input(shape=(original_dim,))
h = inputs
h = layers.Dense(70, activation='relu')(h)
h = layers.Dense(30, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = backend.random_normal(shape=(backend.shape(z_mean)[0], latent_dim),
                                    mean=0., stddev=0.1)
    return z_mean + backend.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

### Define the decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = latent_inputs
x = layers.Dense(30, activation='relu')(x)
x = layers.Dense(70, activation='relu')(x)
x = layers.Dense(vec_length, activation='sigmoid')(x)
outputs = x
decoder = keras.Model(latent_inputs, outputs, name='decoder')

outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

### Define the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - backend.square(z_mean) - backend.exp(z_log_sigma)
kl_loss = backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

Do some training

In [4]:
opt = optimizers.Adamax(learning_rate=0.004)
vae.compile(optimizer=opt, loss='binary_crossentropy')
vae.fit(train_vecs, train_vecs,
        epochs=120,
        batch_size=8,
        shuffle=True,
        validation_data=(val_vecs, val_vecs))

Epoch 1/120
1250/1250 [==============================] - 1s 839us/step - loss: 28.3340 - val_loss: 25.9130
Epoch 2/120
1250/1250 [==============================] - 1s 739us/step - loss: 24.5052 - val_loss: 23.5931
Epoch 3/120
1250/1250 [==============================] - 1s 744us/step - loss: 22.7444 - val_loss: 22.5204
Epoch 4/120
1250/1250 [==============================] - 1s 736us/step - loss: 21.6124 - val_loss: 21.4817
Epoch 5/120
1250/1250 [==============================] - 1s 752us/step - loss: 20.7487 - val_loss: 20.7543
Epoch 6/120
1250/1250 [==============================] - 1s 746us/step - loss: 20.0327 - val_loss: 20.1579
Epoch 7/120
1250/1250 [==============================] - 1s 755us/step - loss: 19.4483 - val_loss: 19.5551
Epoch 8/120
1250/1250 [==============================] - 1s 754us/step - loss: 19.0553 - val_loss: 19.3199
Epoch 9/120
1250/1250 [==============================] - 1s 754us/step - loss: 18.7003 - val_loss: 18.9347
Epoch 10/120
1250/1250 [=============

1250/1250 [==============================] - 1s 775us/step - loss: 8.6474 - val_loss: 9.3049
Epoch 78/120
1250/1250 [==============================] - 1s 736us/step - loss: 8.5933 - val_loss: 9.1973
Epoch 79/120
1250/1250 [==============================] - 1s 693us/step - loss: 8.4859 - val_loss: 8.9417
Epoch 80/120
1250/1250 [==============================] - 1s 755us/step - loss: 8.4712 - val_loss: 8.7790
Epoch 81/120
1250/1250 [==============================] - 1s 736us/step - loss: 8.3758 - val_loss: 8.9412
Epoch 82/120
1250/1250 [==============================] - 1s 693us/step - loss: 8.3266 - val_loss: 8.8840
Epoch 83/120
1250/1250 [==============================] - 1s 744us/step - loss: 8.2347 - val_loss: 8.5544
Epoch 84/120
1250/1250 [==============================] - 1s 765us/step - loss: 8.1853 - val_loss: 8.6376
Epoch 85/120
1250/1250 [==============================] - 1s 733us/step - loss: 8.1195 - val_loss: 8.7665
Epoch 86/120
1250/1250 [==============================] - 1

Evaluation

In [11]:
decoded_regs = vae.predict(test_vecs)
print(test_strs[:10])
recovered = [v.recover(x) for x in decoded_regs]
print(recovered[:10])
acc = np.sum([x == y for x, y in zip(recovered, test_strs)]) / len(test_strs)
print('overall accuracy = ' + str(acc))
print('by digit accuracy:')
for i_digit in range(len(test_strs[0])):
    acc = np.sum([x[i_digit] == y[i_digit] for x, y in zip(recovered, test_strs)]) / len(test_strs)
    print('    ' + str(acc))

['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
['MY33KSZ', 'WE83NRF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89URN', 'GT57AKA', 'CD77PJT']
overall accuracy = 0.575
by digit accuracy:
    0.995
    0.983
    0.99
    0.991
    0.707
    0.865
    0.991


Use decoder as generator

In [6]:
generated_regs = decoder.predict(np.random.random((5, latent_dim)))
print([v.recover(x) for x in generated_regs])

['BS28HIT', 'BJ82VMT', 'RS83NWB', 'FS88YDQ', 'PJ28HJB']


### Avoid generating any training data

There is a chance that a random generation of data might produce one of the actual registration numbers used in training. However, any input registrations are represented as a probability distribution in the latent space; so one could require that any sample vector used in the latent space to be 'sufficiently far' down the tails of all the recorded distributions.



In [7]:
# Get the distributions for each reg number in the data
all_strs = train_strs + val_strs + test_strs
all_vecs = np.array([v.vectorize(x) for x in all_strs])
means, log_sigmas, _ = encoder.predict(all_vecs)
distn_dict = {x: (y,z) for x, y, z in zip(all_strs, means, np.exp(log_sigmas))}  # dict<reg no: (mean, sigma)>
distn_dict['YK66BIQ']
# for a multivariate distribution ~ N(mean, Var) valued in the latent space, 
# mean is the first vector below, and Var is a diagonal matrix, the square root
# of the diagonal entries being given in the second vector.
# If the KL loss function has done its job, the distributions should be close to N(0,1)

(array([-0.5881468 , -0.27519965, -0.19447577, -0.3235448 ,  0.02782166,
         1.3175246 ,  0.07991531, -0.46847036,  0.25594914,  0.76399887,
         0.09407352, -0.08175188,  0.10620986,  0.28594175,  0.17494482],
       dtype=float32),
 array([0.5664883 , 0.5553645 , 0.669898  , 0.6018982 , 0.979184  ,
        0.5987289 , 0.61448604, 0.55870074, 0.62289315, 0.59218544,
        0.5857828 , 0.5573959 , 0.5602451 , 0.5005922 , 0.573779  ],
       dtype=float32))